loading the unique zip codes of our NYC 311 incidents dataset. 

In [1]:
import pandas as pd

cols = list(range(0, 41))
df = pd.read_csv(r'C:\Users\Admin\Desktop\nyc_311_limit_2020.csv', header=None, names=cols)

zips = df.iloc[:, 8]
zips = (x for x in zips if str(x) != 'nan')
unique_zips = set(zips)
# print(unique_zips)

In [3]:
import datetime
import statistics
time = []
t1 = None
t2 = None
reptime = 0

df = df[df[19] == 'Closed']

time_by_zip = {}

for i, x in df.iterrows():
    t1 = datetime.datetime.strptime(str(x[1]), '%m/%d/%Y %I:%M:%S %p')
    t2 = datetime.datetime.strptime(str(x[2]), '%m/%d/%Y %I:%M:%S %p')
    reptime = (t2-t1).total_seconds()/3600
    if x[8] in time_by_zip:
        time_by_zip[x[8]].append((t1, reptime))
    elif str(x[8]) != 'nan':
        time_by_zip[x[8]] = [(t1, reptime)]

#print(time_by_zips)

calculating monthly averages across all the zipcodes.

In [51]:
m_tbz = {}
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep"]

for key in time_by_zip:
    m = {}
    for t, rtime in time_by_zip[key]: #sorting reptimes for each zipcode by month
        month = months[t.month-1]
        if month in m:
            m[month].append(rtime)
        else:
            m[month] = [rtime]
    
    for mo in months:
        if mo not in m:
            m[mo] = ["NA"]

    m_tbz[key] = m

avg_m_tbz = {}

# calculating the average response time, per zipcode, per month
for key in m_tbz: 
    avg_m_tbz[key] = []
    for mo in months:
        if m_tbz[key][mo] != ["NA"]:
            avg_m_tbz[key].append(statistics.mean(m_tbz[key][mo]))
        else:
            avg_m_tbz[key].append(0)

avg_m_tbz

{11234.0: [0, 0, 0, 0, 0, 0, 194.36805555555557, 180.03767676767677, 0],
 11210.0: [0, 0, 0, 0, 0, 0, 386.0939814814815, 162.53527777777776, 0],
 11354.0: [0, 0, 0, 0, 0, 0, 955.9113888888888, 224.05944444444444, 0],
 10451.0: [0,
  1.6883333333333332,
  0.04388888888888889,
  0,
  0,
  14.7,
  0.39381944444444444,
  131.91222222222223,
  0],
 10031.0: [0,
  0,
  0,
  0,
  0,
  42.333333333333336,
  0.14590277777777777,
  24.99173611111111,
  0],
 11364.0: [4110.966666666666,
  0,
  0,
  0,
  0,
  0,
  136.6197222222222,
  266.67291666666665,
  0],
 10301.0: [0, 0, 1.62375, 0, 0, 0, 0, 48.20666666666666, 0],
 10016.0: [0, 0.9766666666666667, 7.166574074074074, 0, 0, 0, 0, 0, 0],
 10305.0: [0,
  4.227222222222222,
  0,
  0,
  0,
  0,
  0.01861111111111111,
  75.46666666666667,
  0],
 11217.0: [0,
  0,
  0.9055555555555556,
  0,
  0,
  0,
  0.2184722222222222,
  11.741666666666667,
  0],
 10022.0: [0, 1.8049305555555555, 0, 0, 0, 0, 0, 0, 0],
 11103.0: [0, 2.1238888888888887, 0, 0, 0, 0,

calculating monthly average response times for all zipcodes

In [52]:
avg_m_2020 = {}
# for every month, add all zipcodes' average reptimes to a list, take the average
for i, mo in enumerate(months):
    allzips = []
    avg_m_2020[mo] = 0
    for key in avg_m_tbz:
        if avg_m_tbz[key][i] != 0: #if there is data for this zipcode for this month
            allzips.append(avg_m_tbz[key][i])
    if allzips != []:
        avg_m_2020[mo] = statistics.mean(allzips)

avg_m_2020

{'Jan': 4110.966666666666,
 'Feb': 1.6811241319444443,
 'Mar': 1.9696373456790124,
 'Apr': 0.5051388888888889,
 'May': 0,
 'Jun': 159.42296296296297,
 'Jul': 39.870056044697975,
 'Aug': 94.84386049898518,
 'Sep': 0}


dropdown = Dropdown(label="Choose Zipcode 1", menu=options)
dropdown2 = Dropdown(label="Choose Zipcode 2", menu=options)
#show(dropdown)

showing line graph of monthly averages

In [65]:
from bokeh.io import show
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.plotting import figure, curdoc

zipcodes = (str(x) for x in unique_zips)
zipcodes = sorted(list(zipcodes))
z1 = Select(title="Choose Zipcode 1", options=zipcodes, value=zipcodes[0])
z2 = Select(title="Choose Zipcode 2", options=zipcodes, value=zipcodes[1])

x = list(i+1 for i in range(9)) # 1-9
y = list(avg_m_2020.values()) # all zipcodes
y1 = avg_m_tbz[float(zipcodes[0])] # zipcode 1, default is 1st zip
y2 = avg_m_tbz[float(zipcodes[1])] # zipcode 2, default is 2nd zip           

ds = ColumnDataSource(data={'x': months, 'y': y, 'y2': y2, 'y1': y1})

all_p = figure(x_range=months, y_range=(0, 200), x_axis_label='Months', y_axis_label='Response Time in Hours')
# x_range=months gives month x-axis tickers
all_p.line(x='x', y='y', line_width=2, line_color='gray', legend_label='All Zipcodes', source=ds)
all_p.line(x='x', y='y1', line_width=2, line_color='red', legend_label='Zipcode 1', source=ds)
all_p.line(x='x', y='y2', line_width=2, line_color='blue', legend_label='Zipcode 2', source=ds)

def change_zip1(attr, old, new): #update zip1
    ds.data={'xs': [x, x, x], 
             'ys': [y, list(avg_m_tbz[z1.value]), y2]}

def change_zip2(attr, old, new): #update zip2
    ds.data={'x': [x, x, x], 
             'y': [y, y1, list(avg_m_tbz[z2.value])]}

z1.on_change('value', change_zip1)
z2.on_change('value', change_zip2)
    
show(all_p)
layout = column(z1, z2, all_p)
#curdoc.add_root(layout)